In [1]:
import pandas as pd
from pymongo import MongoClient

C:\Users\Tay Han\AppData\Local\Temp\ipykernel_16840\577788817.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
user_0 = pd.read_csv('user_df_0.csv')
user_1 = pd.read_csv('user_df_1.csv')
user_2 = pd.read_csv('user_df_2.csv')
user_3 = pd.read_csv('user_df_3.csv')
user_4 = pd.read_csv('user_df_4.csv')
user_5 = pd.read_csv('user_df_5.csv')
user_6 = pd.read_csv('user_df_6.csv')
user_7 = pd.read_csv('user_df_7.csv')
user_8 = pd.read_csv('user_df_8.csv')

In [8]:
print(len(user_0) + len(user_1))
print()

22327



In [10]:
#Stacking the CSVs together

users = pd.concat([user_0,
                user_1, 
                user_2,
                user_3,
                user_4, 
                user_5,
                user_6,
                user_7, 
                user_8])

In [19]:
#Aggregate all duplicate users within each sampling procedure
log = {col: 'mean' if users[col].dtype in ['float64', 'int64'] else 'first' for col in users.columns if col != 'user_id'}

users_result = users.groupby('user_id').agg(log).reset_index()


9329

In [26]:
users_result["label"].value_counts()

label
0                      56808
0                       9329
1                       3657
1                        496
u952295114369654784        1
Name: count, dtype: int64

In [20]:
#Concatenate all tweet relationships

graph_0 = pd.read_csv('graph_df_0.csv')
graph_1 = pd.read_csv('graph_df_1.csv')
graph_2 = pd.read_csv('graph_df_2.csv')
graph_3 = pd.read_csv('graph_df_3.csv')
graph_4 = pd.read_csv('graph_df_4.csv')
graph_5 = pd.read_csv('graph_df_5.csv')
graph_6 = pd.read_csv('graph_df_6.csv')
graph_7 = pd.read_csv('graph_df_7.csv')
graph_8 = pd.read_csv('graph_df_8.csv')

In [24]:
#Stacking the tweet CSVs together

graph = pd.concat([graph_0,
                graph_1, 
                graph_2,
                graph_3,
                graph_4, 
                graph_5,
                graph_6,
                graph_7, 
                graph_8])

In [27]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder

# Create a mapping of user IDs to node indices >> change it up to index
user_ids = users["user_id"]
user_labels = users["label"]
user_to_idx = {user_id: idx for idx, user_id in enumerate(users["user_id"])}

# Create edge index and edge weight tensors
edge_list = []
edge_weight_dict = {}

for _, row in graph.iterrows():
    source_idx = user_to_idx[row['source_user_id']]
    target_idx = user_to_idx[row['target_user_id']]
    idx_pair = (source_idx, target_idx)
    if (idx_pair not in edge_list) :
        edge_list.append(idx_pair)
        edge_weight_dict[idx_pair] = 1
    else:
        edge_weight_dict[idx_pair] += 1

edge_weight_list = list(edge_weight_dict.values())

#force it to be contiguous
edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
edge_weight = torch.tensor(edge_weight_list, dtype=torch.float)

C:\Users\Tay Han\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyError: 255245279

In [31]:
users[users['user_id'] == 255245279]

,user_id,label,_id,created_at,description,id,location,name,profile_image_url,url,...,url_count_tweets,url_ratio,url_max_tweets,time_interval_sd_day,mention_count_tweets,mention_max_tweets,hashtag_count_tweets,hashtags_max_tweets,avg_tweet_length,url_tweet_count
